In [5]:
import math
import numpy as np
import pandas as pd

df = pd.read_feather("firma_data_better.feather")

In [7]:
print("Columns:", df.columns.tolist())
print("Date range:", df['Date'].iloc[0], "to", df['Date'].iloc[-1])

Columns: ['Date', 'Underlying', 'C445', 'C450', 'C455', 'C460', 'C465', 'C470', 'C475', 'C480', 'C485', 'C490', 'C495', 'C500', 'C505', 'C510', 'C515', 'C520', 'P445', 'P450', 'P455', 'P460', 'P465', 'P470', 'P475', 'P480', 'P485', 'P490', 'P495', 'P500', 'P505', 'P510', 'P515', 'P520']
Date range: 2025-09-11 00:00:00 to 2025-10-31 00:00:00


In [8]:
# Choose a strike for the ATM call (here 500, which has no missing prices in the series)
strike = 500
option_col = f"C{strike}"

In [9]:
# Check if the chosen option has any missing values
missing_count = df[option_col].isna().sum()
print(f"Missing data points for strike {strike} call:", missing_count)
# If there are missing prices, we can interpolate or drop those days. Interpolation is chosen here.
if missing_count > 0:
    df[option_col] = df[option_col].interpolate()  # simple linear interpolation for any minor gaps

Missing data points for strike 500 call: 1


In [10]:
# Extract relevant series for underlying and option
S_series = df['Underlying'].values
C_series = df[option_col].values
N = len(C_series)  # number of days in the series
print(f"Using strike {strike} call, total days = {N}")
print(df[['Date', 'Underlying', option_col]].head())
print(df[['Date', 'Underlying', option_col]].tail())

Using strike 500 call, total days = 37
        Date  Underlying  C500
0 2025-09-11      470.73  7.02
1 2025-09-12      471.31   6.0
2 2025-09-15      473.25  7.55
3 2025-09-16      474.32  7.87
4 2025-09-17      473.12  7.22
         Date  Underlying  C500
32 2025-10-27      486.91  0.72
33 2025-10-28      485.77  0.45
34 2025-10-29      485.33  0.39
35 2025-10-30      489.72   0.3
36 2025-10-31      491.88  0.03


In [ ]:

# Cumulative distribution function for standard normal
def norm_cdf(x: float) -> float:
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

# Black-Scholes formula for European call price (no dividends assumed)
def black_scholes_call_price(S: float, K: float, T: float, r: float, sigma: float) -> float:
    """Compute Black-Scholes price of a European call option."""
    if T <= 1e-16:  # effectively at maturity
        return max(0.0, S - K)
    if sigma <= 1e-8:  # near zero volatility
        # If volatility is zero, option value is just discounted intrinsic value (if any)
        return max(0.0, S - K * math.exp(-r * T))
    # Black-Scholes formula components
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    # Call price = S * N(d1) - K * e^{-rT} * N(d2)
    return S * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)

# Black-Scholes Delta for a call option
def black_scholes_delta(S: float, K: float, T: float, r: float, sigma: float) -> float:
    """Compute Black-Scholes delta of a call option."""
    if T <= 1e-16:
        # At maturity, delta is 1 if option finishes in the money, else 0
        return 1.0 if S > K else 0.0
    if sigma <= 1e-8:
        # If essentially zero volatility, delta is 1 if forward S would exceed K, else 0
        return 1.0 if S * math.exp(r * T) > K else 0.0
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    return norm_cdf(d1)

# Black-Scholes Vega for a call option (sensitivity to volatility)
def black_scholes_vega(S: float, K: float, T: float, r: float, sigma: float) -> float:
    """Compute Black-Scholes vega of a call option."""
    if T <= 1e-16 or sigma <= 1e-8:
        return 0.0
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    # Standard normal probability density
    pdf = (1.0 / math.sqrt(2 * math.pi)) * math.exp(-0.5 * d1**2)
    # Vega = S * sqrt(T) * pdf (no dividend case)
    return S * math.sqrt(T) * pdf

# Implied volatility solver using bisection
def implied_volatility_call(C_market: float, S: float, K: float, T: float, r: float=0.0) -> float:
    """Find implied volatility for a call given its market price using bisection."""
    # Handle trivial cases at expiration:
    if T <= 1e-16:
        # If at maturity, implied vol is 0 if price equals intrinsic value
        intrinsic = max(0.0, S - K)
        return 0.0 if abs(C_market - intrinsic) < 1e-8 else float('nan')
    # Initialize bounds for vol
    low_vol, high_vol = 0.0, 5.0  # search between 0% and 500% vol
    # Ensure target price is within bounds of model prices:
    price_low = black_scholes_call_price(S, K, T, r, low_vol + 1e-8)  # near-zero vol price
    price_high = black_scholes_call_price(S, K, T, r, high_vol)
    if C_market < price_low - 1e-8 or C_market > price_high + 1e-8:
        # Price out of feasible range (could happen due to arbitrage or data error)
        return float('nan')
    # Bisection iteration
    for _ in range(100):
        mid_vol = 0.5 * (low_vol + high_vol)
        price_mid = black_scholes_call_price(S, K, T, r, mid_vol)
        if abs(price_mid - C_market) < 1e-6:
            return mid_vol
        if price_mid < C_market:
            low_vol = mid_vol
        else:
            high_vol = mid_vol
    # Return mid of bounds if no exact convergence
    return 0.5 * (low_vol + high_vol)

# Quick test of the functions on the first day of data
r = 0.0  # assume zero risk-free rate for simplicity
S0 = S_series[0]
C0 = C_series[0]
T0 = (N - 1) / 252.0  # initial time to maturity in years (~45 trading days)
imp_vol0 = implied_volatility_call(C0, S0, strike, T0, r)
delta0 = black_scholes_delta(S0, strike, T0, r, imp_vol0)
vega0 = black_scholes_vega(S0, strike, T0, r, imp_vol0)
print(f"Initial implied vol ≈ {imp_vol0:.2%}, Delta ≈ {delta0:.3f}, Vega ≈ {vega0:.2f}")